In [ ]:
### What This Does
This Python script converts a Markdown file (.md) into a Jupyter Notebook file (.ipynb). It automatically creates separate cells in the notebook wherever it finds markdown headings (#, ##, ###, etc.).

### How to Use It
1. Install Requirement: pip install nbformat
2. Run the script: python markdown_to_notebook.py
3. Follow the Prompts text:
    Enter the input markdown file path: xxxx/xxxxx/post-class.md.md
    Enter the output notebook file name (default: output.ipynb): my_notebook.ipynb
4. Result:  Creates my_notebook.ipynb with your markdown content
5. When need to input code in the notebook, just edit the cell type to "Code" in Jupyter Notebook interface.

In [ ]:
import nbformat as nbf
import re

def markdown_to_notebook(input_filepath, output_filepath):
    """
    Convert a markdown file to a Jupyter notebook with markdown cells.
    Split cells based on any markdown heading (#, ##, ###, etc.)
    """
    try:
        with open(input_filepath, 'r', encoding='utf-8') as f:
            markdown_content = f.read()
    except FileNotFoundError:
        print(f"Error: File '{input_filepath}' not found.")
        return
    except Exception as e:
        print(f"Error reading file: {e}")
        return

    # Create a new notebook
    notebook = nbf.v4.new_notebook()
    cells = []
    
    # Find all heading positions
    # This regex finds: start of line (or start of string), 1-6 #, space, then text until end of line
    heading_pattern = r'(^|\n)(#{1,6} .+?)(?=\n#{1,6} |\Z)'
    
    # Add a newline at the beginning to make pattern matching consistent
    content = '\n' + markdown_content
    
    # Find all matches
    matches = list(re.finditer(r'(?=\n#{1,6} )', content))
    
    if not matches:
        # No headings found, create one cell with all content
        if markdown_content.strip():
            cells.append(nbf.v4.new_markdown_cell(markdown_content.strip()))
    else:
        # Process sections between headings
        start_pos = 0
        for match in matches:
            # Get content from start_pos to this heading
            if match.start() > start_pos:
                section = content[start_pos:match.start()].strip()
                if section:
                    # Remove leading newline if present
                    if section.startswith('\n'):
                        section = section[1:]
                    cells.append(nbf.v4.new_markdown_cell(section.strip()))
            
            # Now find where this heading section ends
            heading_start = match.start()
            # Find next heading or end of file
            next_match = re.search(r'(?=\n#{1,6} )', content[match.start()+1:])
            if next_match:
                heading_end = match.start() + 1 + next_match.start()
                section = content[heading_start:heading_end].strip()
                start_pos = heading_end
            else:
                # Last heading goes to end of file
                section = content[heading_start:].strip()
                start_pos = len(content)
            
            if section:
                # Remove leading newline if present
                if section.startswith('\n'):
                    section = section[1:]
                cells.append(nbf.v4.new_markdown_cell(section.strip()))
        
        # Check if there's any content after the last heading
        if start_pos < len(content):
            section = content[start_pos:].strip()
            if section:
                if section.startswith('\n'):
                    section = section[1:]
                cells.append(nbf.v4.new_markdown_cell(section.strip()))

    # Add cells to the notebook
    notebook['cells'] = cells

    # Write the notebook to a file
    try:
        with open(output_filepath, 'w', encoding='utf-8') as f:
            nbf.write(notebook, f)
        print(f"✓ Notebook saved to {output_filepath}")
        print(f"✓ Created {len(cells)} markdown cells")
    except Exception as e:
        print(f"Error writing notebook: {e}")

def main():
    """Main function to run the markdown to notebook converter."""
    # Prompt user for input and output paths
    input_filepath = input("Enter the input markdown file path: ").strip()
    if not input_filepath:
        print("Error: No input file path provided.")
        return
    
    output_filepath = input("Enter the output notebook file name (default: output.ipynb): ").strip()
    if not output_filepath:
        output_filepath = "output.ipynb"
    
    # Add .ipynb extension if not present
    if not output_filepath.endswith('.ipynb'):
        output_filepath += '.ipynb'
    
    # Convert markdown to notebook
    markdown_to_notebook(input_filepath, output_filepath)

if __name__ == "__main__":
    main()